In [1]:
# !pip install textblob

     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
      --------------------------------------- 10.2/636.8 kB ? eta -:--:--
      --------------------------------------- 10.2/636.8 kB ? eta -:--:--
     - ----------------------------------- 30.7/636.8 kB 220.2 kB/s eta 0:00:03
     ---- -------------------------------- 71.7/636.8 kB 357.2 kB/s eta 0:00:02
     -------- --------------------------- 143.4/636.8 kB 711.9 kB/s eta 0:00:01
     ------------ ----------------------- 225.3/636.8 kB 811.5 kB/s eta 0:00:01
     ----------------- ------------------ 307.2/636.8 kB 951.8 kB/s eta 0:00:01
     -------------------------------- ----- 542.7/636.8 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 636.8/636.8 kB 1.5 MB/s eta 0:00:00


In [ ]:
# !pip install vaderSentiment

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from textblob import TextBlob
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
driver = webdriver.Firefox()
driver.get('https://twitter.com/login')

In [3]:
username = driver.find_element(By.XPATH,"//input[@name='text']")
username.send_keys("rynmeister_")
next_button = driver.find_element(By.XPATH,"//span[contains(text(),'Next')]")
next_button.click()

In [4]:
password = driver.find_element(By.XPATH,"//input[@name='password']")
password.send_keys('La=p}7EZHBS.n9G')
log_in = driver.find_element(By.XPATH,"//span[contains(text(),'Log in')]")
log_in.click()

In [11]:
sleep(3)
search_box = driver.find_element(By.XPATH,"//input[@data-testid='SearchBox_Search_Input']")
search_box.send_keys("Alan Walker")
search_box.send_keys(Keys.ENTER)

In [12]:
UserTags=[]
TimeStamps=[]
Tweets=[]
Replys=[]
reTweets=[]
Likes=[]
Views=[]
Sentimen=[]

articles = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")
while True:
    for article in articles:
        UserTag = article.find_element(By.XPATH,".//div[@data-testid='cellInnerDiv']"and".//div[@data-testid='User-Name']"and".//span[contains(text(),'@')]").text
        UserTags.append(UserTag)
        
        TimeStamp = article.find_element(By.XPATH,".//div[@data-testid='cellInnerDiv']"and".//div[@data-testid='User-Name']"and".//time").get_attribute('datetime')
        TimeStamps.append(TimeStamp)
        
        try:
            Tweet = article.find_element(By.XPATH,".//div[@data-testid='tweetText']").text
        except selenium.common.exceptions.NoSuchElementException:
            Tweet = ""
        Tweets.append(Tweet)

        Reply = article.find_element(By.XPATH,".//div[@data-testid='reply']").text
        Replys.append(Reply)
        
        reTweet = article.find_element(By.XPATH,".//div[@data-testid='retweet']").text
        reTweets.append(reTweet)
        
        Like = article.find_element(By.XPATH,".//div[@data-testid='like']").text
        Likes.append(Like)

        try:
            View = article.find_element(By.XPATH,".//div[@data-testid='cellInnerDiv']"and".//div[@role='group']"and".//a[contains(@aria-label, '"+"views"+"')]").text
        except selenium.common.exceptions.NoSuchElementException:
            View = ""
        Views.append(View)

        # sid_obj = SentimentIntensityAnalyzer()
        # sentiment_dict = sid_obj.polarity_scores(Tweet)
        # if sentiment_dict['compound'] >= 0.05 :
        #     value = "1.0"
        # elif sentiment_dict['compound'] <= - 0.05 :
        #     value = "-1.0"
        # else :
        #     value = "0.0"
        clean_tweet = ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",Tweet).split())
        analysis = TextBlob(clean_tweet)
        if analysis.sentiment.polarity > 0.0:
            sentiment = '1.0'
        elif analysis.sentiment.polarity == 0.0:
            sentiment = '0.0'
        else:
            sentiment = '-1.0'
        Sentimen.append(sentiment)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    sleep(3)
    articles = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")
    Tweets2 = list(set(Tweets))
    if len(Tweets2) > 50:
        break

In [13]:
import pandas as pd

df = pd.DataFrame(zip(UserTags,TimeStamps,Tweets,Replys,reTweets,Likes,Views,Sentimen)
                  , columns=['UserTags','TimeStamps','Tweets','Replys','reTweets','Likes','Views','Sentimen'])

df.to_excel ("D:\\tweets_live.xlsx",index=False)

sleep (3)
import os
os.system('start "excel" "D:\\tweets_live.xlsx"')

0